In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [3]:
# IMPORT LIBRARIES

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Starting by entering credentials:

In [5]:
CLIENT_ID = 'B0QDHAKICCF4YA3DKNB5YNOEJNACR5PXFUONV1YB3JIUHWAY' # your Foursquare ID
CLIENT_SECRET = 'QEKV5BXAIX2V55XXTIA1MLGVKGOYW1FKMOUGNE2SYCVYFTL0' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Credentails:
CLIENT_ID: B0QDHAKICCF4YA3DKNB5YNOEJNACR5PXFUONV1YB3JIUHWAY
CLIENT_SECRET:QEKV5BXAIX2V55XXTIA1MLGVKGOYW1FKMOUGNE2SYCVYFTL0


In [6]:
#address = '400 Broad St, Space Needle, Seattle, WA 98109'
address = '400 Broad St, Seattle, WA'
# address = '102 North End Ave, New York, NY'

In [25]:
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Seattle latitude and longitude', latitude, longitude)

Seattle latitude and longitude 47.6205131 -122.34930359883187


School Search Query Defined

In [8]:
search_query = 'School'
radius = 500
print(search_query + ' .... Hello!')

School .... Hello!


Define the corresponding URL

In [9]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=B0QDHAKICCF4YA3DKNB5YNOEJNACR5PXFUONV1YB3JIUHWAY&client_secret=QEKV5BXAIX2V55XXTIA1MLGVKGOYW1FKMOUGNE2SYCVYFTL0&ll=47.6205131,-122.34930359883187&v=20180604&query=School&radius=500&limit=30'

In [10]:
results = requests.get(url).json()


### The search result, initial dataframe before filtering data and modifying columns:

In [20]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,4dd5c8efae60680f14df449e,Pacific Northwest Ballet School - Seattle Phel...,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",v-1589844862,False,301 Mercer St,at 3rd Ave N,47.624239,-122.351470,"[{'label': 'display', 'lat': 47.62423904290061...",445,98109,US,Seattle,WA,United States,"[301 Mercer St (at 3rd Ave N), Seattle, WA 981..."
1,4d9f4be6640b548106a9dd2d,The Center School,"[{'id': '4bf58dd8d48988d13b941735', 'name': 'S...",v-1589844862,False,305 Harrison St,NaN,47.621098,-122.350840,"[{'label': 'display', 'lat': 47.62109838760947...",132,98109,US,Seattle,WA,United States,"[305 Harrison St, Seattle, WA 98109, United St..."
2,4c99509778ffa0930c0e6375,Arthur Murry School Of Dance,"[{'id': '4bf58dd8d48988d134941735', 'name': 'D...",v-1589844862,False,NaN,NaN,47.623852,-122.347428,"[{'label': 'display', 'lat': 47.62385228849827...",397,98109,US,Seattle,WA,United States,"[Seattle, WA 98109, United States]"
3,4db9c674815445c8ca3d0e83,Seattle Public Schools Dept of Athletics,"[{'id': '4bf58dd8d48988d13b941735', 'name': 'S...",v-1589844862,False,400-498 5th Ave N,Harrison St,47.622238,-122.347588,"[{'label': 'display', 'lat': 47.62223807, 'lng...",231,98109,US,Seattle,WA,United States,"[400-498 5th Ave N (Harrison St), Seattle, WA ..."


In [34]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

In [33]:
dataframe_filtered.drop(['crossStreet', 'labeledLatLngs', 'distance', 'cc', 'formattedAddress'], axis =1, inplace = True)
dataframe_filtered

#### Filtered data frame with modified columns. The reader can see droped and modified columns from codes by clicking the toggle shown at the top.

In [23]:
dataframe_filtered.rename(columns = {'lat': 'latitude', 'lng': 'longtitude', 'postalCode': 'postal code'}, inplace = True)
dataframe_filtered                               

,name,categories,address,latitude,longtitude,postal code,city,state,country,id
0,Pacific Northwest Ballet School - Seattle Phel...,Gym,301 Mercer St,47.624239,-122.351470,98109,Seattle,WA,United States,4dd5c8efae60680f14df449e
1,The Center School,School,305 Harrison St,47.621098,-122.350840,98109,Seattle,WA,United States,4d9f4be6640b548106a9dd2d
2,Arthur Murry School Of Dance,Dance Studio,NaN,47.623852,-122.347428,98109,Seattle,WA,United States,4c99509778ffa0930c0e6375
3,Seattle Public Schools Dept of Athletics,School,400-498 5th Ave N,47.622238,-122.347588,98109,Seattle,WA,United States,4db9c674815445c8ca3d0e83


#### Accident area map indicates four schools within radius of 500 meters

In [31]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # Let us generate map centred around the crash area

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map